In [67]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

In [68]:
MAX_PRIORITY = 7 # TODO : read that value in the file directly
RATE = 1e9 / 8 # rate of transmission in the links
OVERHEAD = 42 # bytes

# PROTOTYPE

### Objectives

- Extract data from csv files to put in an appropriate data structure
- Build a network and find shortest path

In [ ]:
streams = pd.read_csv("test_cases/mesh_streams.csv",
                      names=['PCP','StreamName','StreamType','SourceNode','DestinationNode','Size','Period','Deadline']
                     )
topology_cols = [str(i) for i in range(7)]
topology = pd.read_csv("test_cases/mesh_topology.csv",names=topology_cols).groupby('0')

In [70]:
streams.head(10)

,PCP,StreamName,StreamType,SourceNode,DestinationNode,Size,Period,Deadline
0,7,VLAN_0_Flow_0,ATS,node0_0_1_0,node0_0_3_0,100,10000,15000
1,7,VLAN_0_Flow_1,ATS,node0_0_1_1,node0_0_3_1,100,10000,15000
2,7,VLAN_0_Flow_2,ATS,node0_0_1_0,node0_0_3_1,100,10000,15000
3,7,VLAN_0_Flow_3,ATS,node0_0_1_1,node0_0_3_0,100,10000,15000
4,7,VLAN_0_Flow_4,ATS,node0_0_1_0,node0_0_3_0,100,10000,15000
5,7,VLAN_0_Flow_5,ATS,node0_0_1_1,node0_0_3_1,100,10000,15000
6,7,VLAN_0_Flow_6,ATS,node0_0_1_0,node0_0_3_1,100,10000,15000
7,7,VLAN_0_Flow_7,ATS,node0_0_1_1,node0_0_3_0,100,10000,15000
8,7,VLAN_0_Flow_8,ATS,node0_0_1_0,node0_0_3_0,100,10000,15000
9,7,VLAN_0_Flow_9,ATS,node0_0_1_1,node0_0_3_1,100,10000,15000


In [71]:
"""Extracting the data from csv files to DataFrames"""

switches = topology.get_group('SW')
switches = switches.drop(columns=['4','5','6'])
switches.columns = ['DeviceType','DeviceName','Ports','Domain']

end_systems = topology.get_group('ES')
end_systems = end_systems.drop(columns=['4','5','6'])
end_systems.columns = ['DeviceType','DeviceName','Ports','Domain']

links = topology.get_group('LINK')
links.columns = ['LINK','LinkID','SourceDevice','SourcePort','DestinationDevice','DestinationPort','Domain']

In [72]:
end_systems.head()

,DeviceType,DeviceName,Ports,Domain
4,ES,node0_0_1_0,1,NaN
5,ES,node0_0_1_1,1,NaN
6,ES,node0_0_2_0,1,NaN
7,ES,node0_0_2_1,1,NaN
8,ES,node0_0_3_0,1,NaN


In [73]:
switches.head(20)

,DeviceType,DeviceName,Ports,Domain
1,SW,sw_0_1,4,NaN
2,SW,sw_0_2,4,NaN
3,SW,sw_0_3,4,NaN


In [74]:
links.head(20)

,LINK,LinkID,SourceDevice,SourcePort,DestinationDevice,DestinationPort,Domain
10,LINK,e1,sw_0_1,0,sw_0_2,0.0,NaN
11,LINK,e2,sw_0_1,1,sw_0_3,0.0,NaN
12,LINK,e3,sw_0_2,1,sw_0_3,1.0,NaN
13,LINK,e4,sw_0_1,2,node0_0_1_0,1.0,NaN
14,LINK,e5,sw_0_1,3,node0_0_1_1,1.0,NaN
15,LINK,e6,sw_0_2,2,node0_0_2_0,1.0,NaN
16,LINK,e7,sw_0_2,3,node0_0_2_1,1.0,NaN
17,LINK,e8,sw_0_3,2,node0_0_3_0,1.0,NaN
18,LINK,e9,sw_0_3,3,node0_0_3_1,1.0,NaN


In [105]:
class Flow():
    def __init__(self, data_row):
        # filling a class for each stream to manipulate the data more easily
        self.priority = data_row["PCP"]
        self.src = data_row["SourceNode"]
        self.dest = data_row["DestinationNode"]
        self.b = data_row["Size"]
        self.r = data_row["Size"] / data_row["Period"]
        self.deadline = data_row["Deadline"]
        self.name = data_row["StreamName"]
        self.l = data_row["Size"] # packet length
        self.total_delay = None
    
    def find_path(self, G: nx.MultiGraph):
        # first we find the shortest path from the source to the destination of the stream 
        path = nx.shortest_path(G, self.src, self.dest) 
        self.path = path
        graph = nx.path_graph(path)
        edges = graph.edges()
        self.links = []
        for edge in edges:
            link = G.edges.get([edge[0], edge[1], 0])
            self.links.append({"edges": [G.nodes[edge[0]], G.nodes[edge[1]]], "data": link})

    def __repr__(self):
        # just to have a print for comparison with solution.csv file
        path = "->".join(self.path)
        return f"{self.name},{round(self.total_delay * 1e6, 3)},{self.deadline},{path}"

    def fill_nodes(self, G):
        # takes each nodes and adds this flow to the array
        # of all flows going through
        for link in self.links:
            [edge0, edge1] = link["edges"]
            link_id = link["data"]["link_id"]

            '''This was BEFORE i considered only the flows going through the same
            output port'''
            # edge0["output_flows"][self.priority] = [
            #     *edge0["output_flows"].get(self.priority,[]),
            #     self
            # ] # creates an array if it does not exist

            '''In this project we only have one link between two nodes
            therefore, we don't need to look at output ports assignments
            because we have the assignment one port = one link'''
            edge0["output_flows"][self.priority] = edge0["output_flows"].get(self.priority, dict())
            # creates a dictionary if it does not exist
            edge0["output_flows"][self.priority][link_id] = [
                *edge0["output_flows"][self.priority].get(link_id, []),
                self
            ] # creates an array for each link if it does not exist
            
            edge1["input_flows"][self.priority] = edge1["input_flows"].get(self.priority, dict())
            # creates a dictionary if it does not exist
            edge1["input_flows"][self.priority][link_id] = [
                *edge1["input_flows"][self.priority].get(link_id, []),
                self
            ] # creates an array for each link if it does not exist
    def hop_delay(self, link):
        [edge0, edge1] = link["edges"]
        output_flows = edge0["output_flows"]
        input_flows = edge1["input_flows"]
        link_id = link["data"]["link_id"]

        # higher priority 
        bH = 0
        rH = 0
        for priority in range(self.priority+1, MAX_PRIORITY+1):
            for flow in output_flows.get(priority, {}).get(link_id, []):
                bH += flow.b
                rH += flow.r

        # lower priority
        lL = 0 
        for priority in range(0, self.priority):
            for flow in output_flows.get(priority, {}).get(link_id, []):
                if flow.l > lL:
                    lL = flow.l

        # print(f"bH {bH} rH {rH} lL {lL}")
        max_delay = 0
        for flow in input_flows[self.priority][link_id]:
            bc = 0
            for bc_flow in input_flows[self.priority][link_id]:
                if bc_flow != flow:
                    bc += bc_flow.b # we add every flow's burst except for j to bc
            lj = flow.l
            delay = (bH + bc + lL) / (RATE - rH) + lj / RATE + OVERHEAD / RATE
            # print(f"bc {bc} lj {lj}") 
            if delay > max_delay:
                max_delay = delay
        
        return max_delay

    def get_total_delay(self):
        total_delay = 0
        for link in self.links:
            total_delay += self.hop_delay(link)
        self.total_delay = total_delay



In [106]:
G = nx.MultiGraph()
# multi graph to allow multiple links between two same nodes

# this creates the links as well as the necessary nodes
for link in links.iterrows():
    source = link[1]['SourceDevice']
    destination = link[1]['DestinationDevice']
    source_port = link[1]['SourcePort']
    link_id = link[1]['LinkID']
    destination_port = link[1]['DestinationPort']
    G.add_edge(source,
               destination,
               source_port=source_port,
               destination_port=destination_port,
               link_id=link_id,
              )

for end_system in end_systems.iterrows():
    # the names are supposedly already inside the graph because of the link creation
    name = end_system[1]['DeviceName']
    G.nodes[name]['ports'] = end_system[1]['Ports']
    G.nodes[name]['input_flows'] = dict()
    G.nodes[name]['output_flows'] = dict()

for switch in switches.iterrows():
    name = switch[1]['DeviceName']
    G.nodes[name]['ports'] = switch[1]['Ports']
    G.nodes[name]['input_flows'] = dict()
    G.nodes[name]['output_flows'] = dict()

In [107]:
flows = []
for stream in streams.iterrows():
    flow = Flow(stream[1])
    flow.find_path(G)
    flows.append(flow)

for flow in flows:
    flow.fill_nodes(G)
    pass

In [108]:
for flow in flows:
    flow.get_total_delay()
    print(flow)

VLAN_0_Flow_0,23.408,15000,node0_0_1_0->sw_0_1->sw_0_3->node0_0_3_0
VLAN_0_Flow_1,23.408,15000,node0_0_1_1->sw_0_1->sw_0_3->node0_0_3_1
VLAN_0_Flow_2,23.408,15000,node0_0_1_0->sw_0_1->sw_0_3->node0_0_3_1
VLAN_0_Flow_3,23.408,15000,node0_0_1_1->sw_0_1->sw_0_3->node0_0_3_0
VLAN_0_Flow_4,23.408,15000,node0_0_1_0->sw_0_1->sw_0_3->node0_0_3_0
VLAN_0_Flow_5,23.408,15000,node0_0_1_1->sw_0_1->sw_0_3->node0_0_3_1
VLAN_0_Flow_6,23.408,15000,node0_0_1_0->sw_0_1->sw_0_3->node0_0_3_1
VLAN_0_Flow_7,23.408,15000,node0_0_1_1->sw_0_1->sw_0_3->node0_0_3_0
VLAN_0_Flow_8,23.408,15000,node0_0_1_0->sw_0_1->sw_0_3->node0_0_3_0
VLAN_0_Flow_9,23.408,15000,node0_0_1_1->sw_0_1->sw_0_3->node0_0_3_1
VLAN_0_Flow_10,37.008,18000,node0_0_1_0->sw_0_1->sw_0_3->node0_0_3_0
VLAN_0_Flow_11,37.008,18000,node0_0_1_1->sw_0_1->sw_0_3->node0_0_3_1
VLAN_0_Flow_12,37.008,18000,node0_0_1_0->sw_0_1->sw_0_3->node0_0_3_1
VLAN_0_Flow_13,37.008,18000,node0_0_1_1->sw_0_1->sw_0_3->node0_0_3_0
VLAN_0_Flow_14,37.008,18000,node0_0_1_0->sw_

In [89]:
flows[0].get_total_delay()
    # print(f"{flow.name}, {flow.total_delay * 1e6}")
print(flows[0])

VLAN_0_Flow_0,32.816,15000,node0_0_1_0->sw_0_1->sw_0_3->node0_0_3_0


In [83]:
for k, item in G.edges.items():
    print(item)

{'source_port': '0', 'destination_port': 0.0, 'link_id': 'e1'}
{'source_port': '1', 'destination_port': 0.0, 'link_id': 'e2'}
{'source_port': '2', 'destination_port': 1.0, 'link_id': 'e4'}
{'source_port': '3', 'destination_port': 1.0, 'link_id': 'e5'}
{'source_port': '1', 'destination_port': 1.0, 'link_id': 'e3'}
{'source_port': '2', 'destination_port': 1.0, 'link_id': 'e6'}
{'source_port': '3', 'destination_port': 1.0, 'link_id': 'e7'}
{'source_port': '2', 'destination_port': 1.0, 'link_id': 'e8'}
{'source_port': '3', 'destination_port': 1.0, 'link_id': 'e9'}
